# Group profiles

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos'
FOLDER_NAME = 'profiled'
CATALOG_NAME = 'Canarias_municipios.csv'

## Import

In [2]:
from mariachis.models import GroupProfiles

gp = GroupProfiles(BASE_DIR,FOLDER_NAME)
print(gp)

Path:	/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos/profiled
With 7 profiled files


## Transform

In [3]:
df, pipe_obj = gp.full_pipeline(kbest=22, cluster_kwargs={'n_clusters':6, 'kmeans':False}, index='comunity_code', values='cluster', aggfunc=', '.join)
df.sample()

,,Gasto por turista y d√≠a_De 1 a 7 noches,Gasto por turista y d√≠a_Hoteleros,Pernoctaciones,Valor_mean_Bares,Valor_mean_Restaurantes,Valor_sum_Bares,Valor_sum_Restaurantes,Viajeros entrados,count_avg_amount,count_cards,...,mean_cards,median_avg_amount,median_cards,min_cards,sum_avg_amount,valor_max_prec_10,valor_mean_prec_10,valor_median_prec_10,valor_min_prec_10,cluster
comunity_code,cluster_agg,,,,,,,,,,,,,,,,,,,,,
38002,Ambiente Familiar,0.0,0.0,0.0,7.7,9.0,154.0,180.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A


In [4]:
from mariachis.utils import tree_to_code

tree_to_code(df, df.columns)

1.0
  if max_cards <= 22.5:
    if Valor_sum_Restaurantes <= 9433.5:
      return [[128.   0.   0.   0.   0.   0.]]
    else:  # if Valor_sum_Restaurantes > 9433.5
      return [[0. 0. 0. 0. 2. 0.]]
  else:  # if max_cards > 22.5
    if count_cards <= 10488.5:
      return [[0. 0. 0. 4. 0. 0.]]
    else:  # if count_cards > 10488.5
      if median_cards <= 17.0:
        return [[0. 0. 2. 0. 0. 0.]]
      else:  # if median_cards > 17.0
        if max_avg_amount <= 425.5900001525879:
          return [[0. 0. 0. 0. 0. 1.]]
        else:  # if max_avg_amount > 425.5900001525879
          return [[0. 1. 0. 0. 0. 0.]]


## Profiles

In [5]:
from mariachis.utils import profiles

prof = profiles(df)

for i,var in enumerate(prof.values()):
    aux = var.fillna(0)
    display(aux.style.format("{:.0f}").background_gradient('Blues'))
    # aux.T.to_csv(f'Profiles_{i}.csv')

,Gasto por turista y d√≠a_De 1 a 7 noches,Gasto por turista y d√≠a_Hoteleros,Pernoctaciones,Valor_mean_Bares,Valor_mean_Restaurantes,Valor_sum_Bares,Valor_sum_Restaurantes,Viajeros entrados,count_avg_amount,count_cards,max_avg_amount,max_cards,mean_avg_amount,mean_cards,median_avg_amount,median_cards,min_cards,sum_avg_amount,valor_max_prec_10,valor_mean_prec_10,valor_median_prec_10,valor_min_prec_10
cluster,,,,,,,,,,,,,,,,,,,,,,
A,0,0,2819,59,32,1317,667,385,100,100,17,1,3,0,2,0,0,2780,0,0,0,0
B,0,0,0,857,358,23141,9675,0,35748,35748,755,414,28,87,25,20,3,996159,0,0,0,0
C,0,0,0,338,200,9130,5126,0,17346,17346,393,470,38,71,32,14,3,651515,0,0,0,0
D,0,0,0,88,79,2012,1658,0,6108,6108,319,38,33,9,26,8,3,201502,0,0,0,0
E,1,1,164455,819,652,16370,13044,20585,0,0,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,153,56,4133,1123,0,13813,13813,96,67,26,31,25,31,3,365028,0,0,0,0


In [6]:
cluster_dict = {
    'A':'Diversión para toda la familia',
    'B':'La mejor fiesta tropical de tu vida',
    'C':'Ruta del vino, trae tu abrigo',
    'D':'Platillos exclusivos para tus viajes de trabajo',
    'E':'Atrévete a la aventura',
    'F':'Déjate consentir en la zona dorada y el mejor clima',
}

df['final_cluster'] = df['cluster'].map(cluster_dict)
df['final_cluster'].value_counts(1)

Diversión para toda la familia                         0.927536
Platillos exclusivos para tus viajes de trabajo        0.028986
Ruta del vino, trae tu abrigo                          0.014493
Atrévete a la aventura                                 0.014493
La mejor fiesta tropical de tu vida                    0.007246
Déjate consentir en la zona dorada y el mejor clima    0.007246
Name: final_cluster, dtype: float64

In [7]:
import cufflinks as cf
from pandas import DataFrame
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

pipe_pca = Pipeline(steps=[('pre_scaler', MinMaxScaler()), ('dim_red', PCA(3))])
X = DataFrame(pipe_pca.fit_transform(df.iloc[:,:-3]), index=df.index).join(df['final_cluster'])

cf.go_offline()
chart_size = cf.Layout(height=700, width=1100)
X.iplot(kind='scatter3d', mode='markers', x=0, y=1, z=2, categories='final_cluster', layout=chart_size)

## Merge catalog

In [8]:
from pathlib import Path
from pandas import read_csv

cat = read_csv(Path(BASE_DIR).joinpath(CATALOG_NAME))
final = df[['final_cluster']].reset_index().merge(cat)
final.sample(2)

,comunity_code,cluster_agg,final_cluster,country,state,state_code,province,province_code,comunity,lat,lng
1,35002,"Ambiente Familiar, Para ahorrar, Mucho viento",Platillos exclusivos para tus viajes de trabajo,ES,CN,Canarias,GC,Las Palmas,Agüimes,27.952432,-15.485089
54,38023,"Zona Dorada, Clima radical",Diversión para toda la familia,ES,CN,Canarias,TF,Santa Cruz de Tenerife,San Cristóbal de La Laguna,28.492613,-16.329504


## Export

In [9]:
gp.export_result(final, 'grouped_3D.csv')

Exported succesfully!
File:	grouped_3D.csv
Path:	/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos
